In [10]:
import pylhe
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import pandas as pd

In [11]:
lhe_file = "/users/eleves-b/2022/baptiste.barthe-gold/Documents/Comput_HEP/MG5_aMC_v2_9_22/VBF-cW-2/Events/run_01/unweighted_events.lhe.gz"
events = pylhe.read_lhe_with_attributes(lhe_file)
nevents = pylhe.read_num_events(lhe_file)
print(f"Number of events: {nevents}")

Number of events: 10000


In [12]:
def extract_4m(events):
    cross_section_weight, quarks_4m, leptons_4m = [], [], []

    for event in events:
        # select only final state particles
        part = [i for i in event.particles if i.status == 1.0]

        # select only quarks up to charm
        quarks = [i for i in part if abs(i.id) <= 4]

        # select only leptons
        leptons = [i for i in part if abs(i.id) in [11, 13, 15]]

        # sanity check, we expect quarks and leptons to be a list of two elements
        assert len(quarks) == 2, f"Length of selected quarks is not 2: {len(quarks)}"
        assert len(leptons) == 2, f"Length of selected leptons is not 2: {len(leptons)}"

        quarks_4m.append(
            [
                {
                    "px": quarks[0].px,
                    "py": quarks[0].py,
                    "pz": quarks[0].pz,
                    "e": quarks[0].e,
                },
                {
                    "px": quarks[1].px,
                    "py": quarks[1].py,
                    "pz": quarks[1].pz,
                    "e": quarks[1].e,
                },
            ]
        )

        leptons_4m.append(
            [
                {
                    "px": leptons[0].px,
                    "py": leptons[0].py,
                    "pz": leptons[0].pz,
                    "e": leptons[0].e,
                },
                {
                    "px": leptons[1].px,
                    "py": leptons[1].py,
                    "pz": leptons[1].pz,
                    "e": leptons[1].e,
                },
            ]
        )

        cross_section_weight.append(event.eventinfo.weight)

    quarks_4m = ak.Array(quarks_4m, with_name="Momentum4D")
    leptons_4m = ak.Array(leptons_4m, with_name="Momentum4D")

    return quarks_4m, leptons_4m, cross_section_weight

In [13]:
quarks_4m, leptons_4m, cross_section_weight = extract_4m(events)

In [14]:
dataset = pd.DataFrame(
    {
        "m_ll": (leptons_4m[:, 0] + leptons_4m[:, 1]).mass,
        "m_jj": (quarks_4m[:, 0] + quarks_4m[:, 1]).mass,
        "pt_l1": leptons_4m[:, 0].pt,
        "pt_l2": leptons_4m[:, 1].pt,
        "pt_j1": quarks_4m[:, 0].pt,
        "pt_j2": quarks_4m[:, 1].pt,
        "pt_ll": (leptons_4m[:, 0] + leptons_4m[:, 1]).pt,
        "eta_l1": leptons_4m[:, 0].eta,
        "eta_l2": leptons_4m[:, 1].eta,
        "eta_j1": quarks_4m[:, 0].eta,
        "eta_j2": quarks_4m[:, 1].eta,
        "delta_eta_jj": quarks_4m[:, 0].eta - quarks_4m[:, 1].eta,
        "delta_phi_jj": quarks_4m[:, 0].phi - quarks_4m[:, 1].phi,
    }
)

In [15]:
dataset.head()

,m_ll,m_jj,pt_l1,pt_l2,pt_j1,pt_j2,pt_ll,eta_l1,eta_l2,eta_j1,eta_j2,delta_eta_jj,delta_phi_jj
0,226.393438,1193.405290,111.980931,988.699367,1023.478769,199.066110,1095.085498,-0.369683,0.215219,0.880640,-1.121572,2.002212,1.292158
1,102.200589,683.618664,56.477553,174.626744,231.541112,42.440512,230.592560,1.497241,0.519281,0.562082,4.418771,-3.856689,-1.684997
2,97.522113,738.529889,781.796595,33.783846,802.819856,56.877340,813.224539,0.301413,0.760824,-0.363627,-2.861775,2.498148,-4.861636
3,93.579285,582.739226,100.955526,146.874370,204.684022,41.492955,232.836333,-1.400548,-1.722492,0.583531,-3.135093,3.718624,-0.897220
4,91.844733,476.403192,301.006567,68.186255,330.692616,57.448325,359.027439,1.455244,1.231388,-0.007349,2.536069,-2.543417,1.129079


In [16]:
dataset.to_csv("./data/cW_2_10k.csv", index=False)